In [1]:
import numpy as np
from matplotlib import pyplot as plt

# 23.6

In [2]:
def my_LQG(A,B,H,Q,M,V,W,R,x0,T):
    S = [None]*(T+1)
    P = [None]*(T+1)
    #setup the LQR gain here
    P[T] = M
    for k in range(T, -1, -1):
        var = R+(B.T@P[k])@B
        P[k-1] = (A.T@P[k])@A + Q - (A.T@P[k]@B)@np.linalg.pinv(var)@(B.T@P[k]@A)

    u = [None]*T
    K = [None]*(T+1)
    L = [None]*T
    S = [None]*(T+1)
    S[0] = np.eye(2)
    #setup the Kalman gain
    for k in range(1,T):
        S[k] = A@(S[k-1]-S[k-1]@H.T@np.linalg.pinv(H@S[k-1]@H.T+ V)@H@S[k-1])@A.T+W
    #now compute both gain matrices at each step
    #note that np.squeeze is only used because this term is - actually a scalar...
    K[0] = -B.T@P[1]@A/(np.squeeze(B.T@P[1]@B)+R)
    for k in range(1,T):
        K[k] = -B.T@P[k+1]@A/(np.squeeze(B.T@P[k+1]@B)+R)
        L[k] = S[k-1]@H.T@np.linalg.pinv(H@S[k-1]@H.T+V)
    full_state = [None]*(T+1)
    full_state[0] = np.array([x0]).T
    observed_state = [None]*(T+1)
    #finally we are ready to compute the solution.
    u[0] = K[0]@full_state[0]
    for k in range(1,T):
        full_state[k] = A@full_state[k-1]+B@u[k-1] + np.random.multivariate_normal(np.zeros(len(W)), cov=W)
        observed_state[k] = H@full_state[k] + np.random.multivariate_normal(np.zeros(len(V)), cov=V)
        u[k] = -K[k]@observed_state[k]
    return u, observed_state, full_state

# 23.7

In [13]:
A = np.array([[1,1],[0,1]])
B = np.array([[0],[1]])
H = np.diag([1,0])
Q, M = np.eye(2), np.eye(2)
V, W = 0.1*np.eye(2), 0.1*np.eye(2)
x0 = np.array([1,0])
R = 1

u, obs, state = my_LQG(A, B, H, Q, M, V, W, R, x0, 100)
obs

[None,
 array([[ 1.21498731,  1.01350244],
        [ 0.12268077, -0.01106337]]),
 array([[ 0.51269415,  0.95925342],
        [-0.16342471,  0.11728814]]),
 array([[ 0.92172817,  0.7640169 ],
        [-0.06416081, -0.09845048]]),
 array([[ 1.28232088,  1.46891158],
        [-0.01096407, -0.00584565]]),
 array([[ 1.63238315,  2.34631427],
        [-0.18215543,  0.09744371]]),
 array([[ 2.9247855 ,  3.5352153 ],
        [-0.18115704, -0.0828625 ]]),
 array([[ 4.92533597,  6.36299148],
        [-0.01063416,  0.16420095]]),
 array([[ 7.55140081e+00,  1.00410050e+01],
        [-1.28301443e-03, -1.87199416e-02]]),
 array([[12.18128859, 16.80958759],
        [-0.06392357, -0.0481311 ]]),
 array([[20.10665266, 27.84140298],
        [ 0.07462759, -0.09177901]]),
 array([[3.28479496e+01, 4.62531863e+01],
        [1.05549528e-01, 3.33749984e-02]]),
 array([[53.92568379, 76.11238393],
        [-0.19365201,  0.14114186]]),
 array([[ 8.92802153e+01,  1.25433565e+02],
        [-1.22028719e-01,  9.8745